# Os dados

Para coletar os dados de outros países, foram utilizadas as APIs dos sites:

- [About Corona](https://about-corona.net/dashboard/)
- [Europe RestFul](https://restcountries.eu/)

In [ ]:

import requests
import pandas as pd

covid_api = 'https://corona-api.com/countries/'
rest_countries = 'https://restcountries.eu/rest/v2/alpha/'
country = 'BR' # Alpha-2 ISO3166

data_json =  requests.get(covid_api + country).json()
country = requests.get(covid_api + country).json()

N = country['data']['population']

print(country['data']['name'])


### Organizando os dados

In [ ]:

from datetime import datetime

df = pd.DataFrame(data_json['data']['timeline'])
df = df.sort_values('date').reset_index()

from datetime import datetime, timedelta
df['date'] = [datetime.fromisoformat(f) for f in df['date']]
df = df.drop_duplicates(subset='date', keep = 'last')

# Criando o vetor de tempo
first_date = df['date'].iloc[0]
size_days = (df['date'].iloc[-1] - df['date'].iloc[0]).days
date_vec = [first_date + timedelta(days=k) for k in range(size_days)]

new_df = pd.DataFrame(date_vec, columns=['date'])
new_df = pd.merge(new_df, df, how='left', on='date')
new_df = new_df.drop(columns= ['index',  'updated_at', 'is_in_progress'])

for col in new_df.columns[1:]:
    new_df[col] = new_df[col].interpolate(method='polynomial', order=1)
df = new_df.dropna()

df


In [ ]:
df.iloc[135,:] = [df.iloc[135, 0]] + [None]*7
df = df.interpolate(method ='linear', limit_direction ='forward')
df = df.where(df.active != 0.0).dropna()

df

In [ ]:
df["active"].dtype==np.float64

### Visualizando os dados

In [ ]:

import numpy as np
from datetime import datetime, timedelta

# Creating the time vector for plotly
datetime_64 = df["date"].values
ts = (datetime_64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
time = [datetime.utcfromtimestamp(t) for t in ts]


In [ ]:

import plotly.graph_objects as go

first = -1

figure = go.Figure()
figure.add_trace(go.Scatter(
                    x=time[:first], 
                    y=df["deaths"].values[:first],
                    mode='lines',
                    line_shape='spline',
                    name = "Mortes",
                    line = dict(color='#e91e63', width=3)))
figure.add_trace(go.Scatter(
                    x=time[:first], 
                    y=df["confirmed"].values[:first],
                    mode='lines',
                    line_shape='spline',
                    name = "Infectados acumulados",
                    line = dict(color='#03a9f4', width=3, dash="dot")))
figure.add_trace(go.Scatter(
                    x=time[:first], 
                    y=df["recovered"].values[:first],
                    mode='lines',
                    line_shape='spline',
                    name = "Recuperados",
                    line = dict(color='#76d275', width=3)))
figure.add_trace(go.Scatter(
                    x=time[:first], 
                    y=df["active"].values[:first],
                    mode='lines',
                    line_shape='spline',
                    name = "Infectados ativos",
                    line = dict(color='#ffca28', width=3)))

figure.update_layout(template='plotly_dark',
                  xaxis_title='Data',
                  yaxis_title='Indivíduos',
                  title_text="Valores iniciais de infectados - " + country['data']['name'])
# 'ggplot2', 'seaborn', 'simple_white', 'plotly',
# 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
# 'ygridoff', 'gridon', 'none'

figure.show()

## Verificando os dados

In [ ]:

dif_I = np.diff(df['active'])
cum = []
cum.append(dif_I[0])

for k, i in enumerate(dif_I):
    cum.append(cum[-1] + i)
    
cum = np.array(cum)
cum += df['deaths'].to_numpy() + df['recovered'].to_numpy()

print("Erro entre casos acumulados e valores de confirmados: {}".format(
        round(sum((cum - df['confirmed'].values)**2 / len(cum)),2) ) ) 


## Encontrando o início da epidemia

Encontrando o início da epidemia para eliminar dados não informativos.

In [ ]:

# Moment where cases are bigger than 2000...
start_moment = np.argmax(df["active"].to_numpy() >= 2000)


## Criando os dados SIR

In [ ]:

time_ref = time[start_moment:]
I = df['active'].to_numpy()[start_moment:]
R = df['recovered'].to_numpy()[start_moment:]
M = df['deaths'].to_numpy()[start_moment:]
S = N - R - I

# Creating the time vector
t = np.linspace(0, len(I)-1, len(I))

Sd, Id, Md, Rd, td = S, I, M, R, t


In [ ]:
Ro = (np.log(S[0]/N) - np.log(S[-1]/N))/(S[0]/N - S[-1]/N)
print("Calculo do Ro baseado nos dados:", Ro)

# Estimando utilizando todos os dados

In [ ]:

from models import *

dataset = dict(S=Sd, I=Id, R=Rd)

# Create the model
sir_model = ss.SIR(pop=N, focus=["S", "I", "R"])

# Adjust the parameters
sir_model.fit(dataset, td,  
              search_pop=True,
              pop_sens=[0.00001, 0.05],
              Ro_sens=[0.8, 15], 
              D_sens=[5, 40])


# Predict the model
initial = (Sd[0], Id[0], Rd[0])
tsim = np.linspace(0, len(I)+120-1, len(I)+120)
time_ref_sim = [time_ref[0] + timedelta(days=i) for i in tsim]

sim_res = sir_model.predict(initial, tsim)


In [ ]:

print("Parâmetros estimados: ", sir_model.parameters)
print("Suposto Ro: ", sir_model.parameters[0])
print("Valores de beta: {}, e r: {}".format(
    sir_model.parameters[0]/sir_model.parameters[1], 1/sir_model.parameters[1]))
print("Dias contaminados: ", sir_model.parameters[1])


In [ ]:

figure = go.Figure()
figure.add_trace(go.Scatter(
                    x=time_ref, 
                    y=I,
                    name="Infectados",
                    mode='lines',
                    line_shape='spline',
                    line = dict(color='#ffca28', width=3)))
figure.add_trace(go.Scatter(
                    x=time_ref,
                    y=R,
                    name="Recuperados",
                    mode="lines",
                    line_shape="spline",
                    line=dict(color='#76d275', width=3)))
figure.add_trace(go.Scatter(
                    x=time_ref_sim,
                    y=sim_res[0],
                    name="Suscetíveis - modelo",
                    mode="lines",
                    line_shape="spline",
                    line=dict(color="#1e88e5", width=4, dash="dash")))
figure.add_trace(go.Scatter(
                    x=time_ref_sim,
                    y=sim_res[2],
                    name="Recuperados - modelo",
                    mode="lines",
                    line_shape="spline",
                    line=dict(color="#43a047", width=4, dash="dash")))
figure.add_trace(go.Scatter(
                    x=time_ref_sim,
                    y=sim_res[1],
                    name="Infectados - modelo",
                    mode="lines",
                    line_shape="spline",
                    line=dict(color="#f4511e", width=4, dash="dash")))
figure.update_layout(template='plotly_dark',
                  xaxis_title='Data', yaxis_title='Indivíduos',
                  title_text="Valores iniciais de infectados - " + country['data']['name'])
# 'ggplot2', 'seaborn', 'simple_white', 'plotly',
# 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
# 'ygridoff', 'gridon', 'none'
figure.show()


# Monte Carlo


Nesta parte, faremos um teste aumentando a quantidade de amostras de treinamento e prevendo o momento do pico da epidemia a medida que mais dias são utilizados para treinamento. Esse estudo vai possibilitar a análise da certeza da previsão do pico da epidemia antes desse acontecer.

In [ ]:

saved_param = {'r':[], 'beta':[], 'pop':[]}
saved_prediction = []

start_day = 10
pred_t = np.array(range(int(td[-1])))

for i in range(start_day, len(I)):

    dataset = dict(S=Sd[:i], I=Id[:i], R=Rd[:i])
    td_ = td[:i]

    # Create the model
    sir_model = ss.SIR(pop=N, focus=["S", "I", "R"], verbose = False)

    # Adjust the parameters
    sir_model.fit(dataset, td_, 
                  search_pop=True,
                  pop_sens=[0.00001,0.05],
                  Ro_sens=[0.8,15], 
                  D_sens=[5,40])
    
    saved_param['beta'].append(sir_model.parameters[0])
    saved_param['r'].append(sir_model.parameters[1])
    saved_param['pop'].append(sir_model.parameters[2])
    
    saved_prediction.append(sir_model.predict((Sd[0],Id[0], Rd[0]), pred_t))
    

In [ ]:
import pickle

with open("./China_II_mc_runs.pickle", "wb") as handle:
    pickle.dump({"pars":saved_param, "pred":saved_prediction}, handle)
    

# Análise do uso do sistema de saúde

Nesta análise, mostramos o erro percentual do quanto antes do pico, conseguimos prever a quantidade de pessoas que realmente serão identificadas como infectadas, uma vez que $R(\infty)$ é a quantidade de pessoas recuperadas totais, daquelas que foram noficadas como infectadas no sistema de saúde. Desta forma segue o erro proporcional do erro a medida em que novos dados diários foram incluidos no modelo:

In [ ]:

x = range(start_day, len(I))
usage_error = [ 100 * abs(p*N - Rd[-1]) / Rd[-1] for p in saved_param['pop']]

fig9 = go.Figure()

fig9.add_trace(go.Scatter(
                    x=td[start_day:peak_pos], 
                    y=usage_error[:peak_pos-start_day],
                    mode='lines',
                    name='Antes do pico',
                    line_shape='spline',
                    line = dict(color='royalblue', width=3),
                    hovertemplate="ε(%) = %{y:.0f}, <br> com %{x:.0f} dias de dados."))
fig9.add_trace(go.Scatter(
                    x=td[peak_pos:-1], 
                    y=usage_error[peak_pos-start_day:-1],
                    mode='lines',
                    line_shape='spline',
                    name='Depois do pico',
                    line = dict(color='royalblue', width=3, dash='dot'),
                    hovertemplate="ε(%) = %{y:.0f}, <br> com %{x:.0f} dias de dados."))
fig9.add_trace(go.Scatter(
                    mode="markers", x=[peak_pos], y=[usage_error[peak_pos-start_day]], 
                    marker_symbol="hexagram-dot", name="Momento do pico",
                    marker_line_color="midnightblue", marker_color="lightskyblue", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="Pico no dia %{x}, com um ε(%) = %{y:.0f}."))
fig9.add_trace(go.Scatter(
                    mode="markers", x=[td[-1]], y=[usage_error[-1]], 
                    marker_symbol="triangle-right-dot", name="Valor atual",
                    marker_line_color="#a00037", marker_color="#ff5c8d", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="No dia %{x} da epidemia, <br> convergindo para ε(%) = %{y:.4f}."))
fig9.update_layout(template='xgridoff', yaxis_range=[-1,100],
                  legend_orientation="h", legend=dict(x=0.10, y=1.05),
                  xaxis_title='Dias (desde o começo da epidemia) utilizados para treinamento',
                  yaxis_title='Erro (%) do R(∞) - notificados',
                  title_text="Erro da previsão do uso do sistema de saúde - 🇨🇳 " + country['data']['name'])

fig9.show(renderer="png")


# Visualizando as previsões de $I(t)$

Vamos analisar as previsões quando somente os dados antes do pico são fornecidos ao modelo, e as previões utilizando os dados após o pico:

In [ ]:

visual_peak = 30

p2 = figure(plot_height=500,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Evolução do COVID - " + country['data']['name'])

# Preparando o estilo
p2.grid.grid_line_alpha = 0
p2.ygrid.band_fill_color = "olive"
p2.ygrid.band_fill_alpha = 0.1
p2.yaxis.axis_label = "Indivíduos"
p2.xaxis.axis_label = "Dias"

# Incluindo as curvas
p2.line(td, Id,
       legend_label="Infectados", 
       line_cap="round", line_width=3, color="#c62828")

for data in saved_prediction[visual_peak-start_day:]:
    p2.line(pred_t[:len(td)], -data[1][:len(td)],
           legend_label="Previsão Infectados - Depois do pico", 
           line_cap="round", line_dash="dashed", line_width=4, color="#ffa000", line_alpha = 0.1)

for data in saved_prediction[:visual_peak-start_day]:
    p2.line(pred_t[:len(td)], -data[1][:len(td)],
           legend_label="Previsão Infectados - Antes do pico", 
           line_cap="round", line_dash="dashed", line_width=4, color="#42a5f5", line_alpha = 0.3)

# Colocando as legendas
p2.legend.click_policy="hide"
p2.legend.location = "top_right"

show(p2)


# Visualizando os ajustes dos grupos


Aqui vamos analisar as previsões obtidas para cada grupo $S(t)$, $I(t)$ e $R(t)$, a medida que mais dias foram informados ao modelo.

In [ ]:

p3 = figure(plot_height=350,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Evolução do COVID - I(t) e R(t) - " + country['data']['name'])

p4 = figure(plot_height=350,
           plot_width=600, 
           tools="", 
           toolbar_location=None,
           title="Evolução do COVID - S(t) - " + country['data']['name'])

plot_all = True

# Preparando o estilo
p3.grid.grid_line_alpha = 0
p3.ygrid.band_fill_color = "olive"
p3.ygrid.band_fill_alpha = 0.1
p3.yaxis.axis_label = "Indivíduos"
p3.xaxis.axis_label = "Dias"

p4.grid.grid_line_alpha = 0
p4.ygrid.band_fill_color = "olive"
p4.ygrid.band_fill_alpha = 0.1
p4.yaxis.axis_label = "Indivíduos"
p4.xaxis.axis_label = "Dias"

# Incluindo as curvas
for data in saved_prediction[20:]:
    p3.line(pred_t, data[1],
           legend_label="Previsão Infectados", 
           line_cap="round", line_dash = 'dashed', 
           line_width=4, color="#42a5f5", line_alpha = 0.1)
    
    p3.line(pred_t, data[2],
           legend_label="Previsão Recuperados", 
           line_cap="round", line_dash = 'dashed', line_width=4, 
           color="#9c27b0", line_alpha = 0.07)
    
    
p3.line(td, Id,
       legend_label="Infectados", 
       line_cap="round", line_width=3, color="#005cb2")

p3.line(td, Rd,
           legend_label="Recuperados", 
           line_cap="round", line_width=3, color="#5e35b1")

if plot_all:
    for data in saved_prediction:
        p4.line(pred_t, data[0] + N*(1-saved_param['pop'][-1]),
               legend_label="Previsão Suscetiveis", 
               line_cap="round", line_dash = 'dashed', 
               line_width=4, color="#ff5722", line_alpha = 0.07)
    p4.line(td, Sd,
           legend_label="Suscetiveis", 
           line_cap="round", line_width=3, color="#b71c1c")

# Colocando as legendas
p3.legend.click_policy="hide"
p3.legend.location = "top_left"

p4.legend.click_policy="hide"
p4.legend.location = "top_right"

show(column(p3,p4))


# Análise de variação do $R_0$

In [ ]:
x = range(start_day, len(I))
beta_norm = [b*p for b,p in zip(saved_param['beta'],saved_param['pop'])]
Ro = [b*p/r for b,r,p in zip(saved_param['beta'],saved_param['r'],saved_param['pop'])]


fig1 = go.Figure()

fig1.add_trace(go.Scatter(
                    x=td[start_day:peak_pos], 
                    y=beta_norm[:peak_pos-start_day],
                    mode='lines',
                    name='Antes do pico',
                    line_shape='spline',
                    line = dict(color='royalblue', width=3),
                    hovertemplate="β = %{y:.4f}, <br> com %{x:.0f} dias de dados."))
fig1.add_trace(go.Scatter(
                    x=td[peak_pos:-1], 
                    y=beta_norm[peak_pos-start_day:-1],
                    mode='lines',
                    line_shape='spline',
                    name='Depois do pico',
                    line = dict(color='royalblue', width=3, dash='dot'),
                    hovertemplate="β = %{y:.4f}, <br> com %{x:.0f} dias de dados."))
fig1.add_trace(go.Scatter(
                    mode="markers", x=[peak_pos], y=[beta_norm[peak_pos-start_day]], 
                    marker_symbol="hexagram-dot", name="Momento do pico",
                    marker_line_color="midnightblue", marker_color="lightskyblue", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="Pico no dia %{x}, com um β = %{y:.4f}."))
fig1.add_trace(go.Scatter(
                    mode="markers", x=[td[-1]], y=[beta_norm[-1]], 
                    marker_symbol="triangle-right-dot", name="Valor atual",
                    marker_line_color="#a00037", marker_color="#ff5c8d", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="No dia %{x} da epidemia, <br> convergindo para β = %{y:.4f}."))
fig1.update_layout(template='xgridoff',
                  legend_orientation="h", legend=dict(x=0.10, y=1.05),
                  xaxis_title='Dias (desde o começo da epidemia) utilizados para treinamento',
                  yaxis_title='Parâmetro β',
                  title_text="Estimativa do parâmetro β - 🇨🇳 " + country['data']['name'])

fig1.show(renderer="png")

In [ ]:

fig2 = go.Figure()

fig2.add_trace(go.Scatter(
                    x=td[start_day:peak_pos], 
                    y=saved_param['r'][:peak_pos-start_day],
                    mode='lines',
                    name='Antes do pico',
                    line_shape='spline',
                    line = dict(color='royalblue', width=3),
                    hovertemplate="r = %{y:.4f}, <br> com %{x:.0f} dias de dados."))
fig2.add_trace(go.Scatter(
                    x=td[peak_pos:-1], 
                    y=saved_param['r'][peak_pos-start_day:-1],
                    mode='lines',
                    line_shape='spline',
                    name='Depois do pico',
                    line = dict(color='royalblue', width=3, dash='dot'),
                    hovertemplate="r = %{y:.4f}, <br> com %{x:.0f} dias de dados."))
fig2.add_trace(go.Scatter(
                    mode="markers", x=[peak_pos], y=[saved_param['r'][peak_pos-start_day]], 
                    marker_symbol="hexagram-dot", name="Momento do pico",
                    marker_line_color="midnightblue", marker_color="lightskyblue", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="Pico no dia %{x}, com um r = %{y:.4f}."))
fig2.add_trace(go.Scatter(
                    mode="markers", x=[td[-1]], y=[saved_param['r'][-1]], 
                    marker_symbol="triangle-right-dot", name="Valor atual",
                    marker_line_color="#a00037", marker_color="#ff5c8d", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="No dia %{x} da epidemia, <br> convergindo para r = %{y:.4f}."))
fig2.update_layout(template='xgridoff',
                  legend_orientation="h", legend=dict(x=0.07, y=1.06),
                  xaxis_title='Dias (desde o começo da epidemia) utilizados para treinamento',
                  yaxis_title='Parâmetro r',
                  title_text="Estimativa do parâmetro r - 🇨🇳 " + country['data']['name'])

fig2.show(renderer="png")


In [ ]:

fig3 = go.Figure()

fig3.add_trace(go.Scatter(
                    x=td[start_day:peak_pos], 
                    y=Ro[:peak_pos-start_day],
                    mode='lines',
                    name='Antes do pico',
                    line_shape='spline',
                    line = dict(color='royalblue', width=3),
                    hovertemplate="r = %{y:.4f}, <br> com %{x:.0f} dias de dados."))
fig3.add_trace(go.Scatter(
                    x=td[peak_pos:-1], 
                    y=Ro[peak_pos-start_day:-1],
                    mode='lines',
                    line_shape='spline',
                    name='Depois do pico',
                    line = dict(color='royalblue', width=3, dash='dot'),
                    hovertemplate="Ro = %{y:.4f}, <br> com %{x:.0f} dias de dados."))
fig3.add_trace(go.Scatter(
                    mode="markers", x=[peak_pos], y=[Ro[peak_pos-start_day]], 
                    marker_symbol="hexagram-dot", name="Momento do pico",
                    marker_line_color="midnightblue", marker_color="lightskyblue", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="Pico no dia %{x}, com um Ro = %{y:.4f}."))
fig3.add_trace(go.Scatter(
                    mode="markers", x=[td[-1]], y=[Ro[-1]], 
                    marker_symbol="triangle-right-dot", name="Valor atual",
                    marker_line_color="#a00037", marker_color="#ff5c8d", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="No dia %{x} da epidemia, <br> convergindo para Ro = %{y:.4f}."))
fig3.update_layout(template='xgridoff',
                  legend_orientation="h", legend=dict(x=0.07, y=1.06),
                  xaxis_title='Dias (desde o começo da epidemia) utilizados para treinamento',
                  yaxis_title='Parâmetro Ro',
                  title_text="Estimativa do parâmetro Ro - 🇨🇳 " + country['data']['name'])

fig3.show(renderer="png")


# Análise de confiança

In [ ]:

from PyAstronomy import pyasl

dI = np.gradient(pyasl.smooth(I, 13, "hamming"))
t = np.linspace(0, len(dI), len(dI))

signal = np.array([di >= 0 for di in dI[::-1]])


import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
                    x=t[signal], 
                    y=dI[::-1][signal],
                    mode='lines',
                    name='Antes do pico - Derivada positiva',
                    line_shape='spline',
                    line = dict(color='#512da8', width=3)))
fig.add_trace(go.Scatter(
                    x=t[~signal], 
                    y=dI[::-1][~signal],
                    mode='lines',
                    line_shape='spline',
                    name='Depois do pico - Derivada negativa',
                    line = dict(color='#d8576b', width=3)))

fig.update_layout(template='xgridoff',
                  xaxis_title='Dias (invertido)',
                  yaxis_title='Taxa de variação de indivíduos',
                  title_text="Derivada da curva de Infectados - " + country['data']['name'])
# 'ggplot2', 'seaborn', 'simple_white', 'plotly',
# 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
# 'ygridoff', 'gridon', 'none'

fig.show(renderer="png")


In [ ]:

peak_pos = np.argmax(~signal[::-1])
print("O pico da epidemia acontece no dia", peak_pos)

final_day = int(td[-1])

estimated_peaks = []
for data in saved_prediction:
    dI = np.gradient(data[1][:final_day])
    signal_pred = np.array([di >= 0 for di in dI[::-1]])
    estimated_peaks.append(len(Id) - np.argmax(signal_pred))
estimated_peaks = np.array(estimated_peaks)


In [ ]:

import plotly.graph_objects as go

peak_error = np.abs(estimated_peaks - peak_pos)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(
                    x=td[start_day:peak_pos], 
                    y=peak_error[:peak_pos-start_day],
                    mode='lines',
                    name='Antes do pico',
                    line_shape='spline',
                    line = dict(color='royalblue', width=3),
                    hovertemplate="Erro de %{y} dias, <br> com %{x:.0f} dias de dados."))
fig1.add_trace(go.Scatter(
                    x=td[peak_pos:], 
                    y=peak_error[peak_pos-start_day:],
                    mode='lines',
                    line_shape='spline',
                    name='Depois do pico',
                    line = dict(color='royalblue', width=3, dash='dot'),
                    hovertemplate="Erro de %{y} dias, <br> com %{x:.0f} dias de dados."))
fig1.add_trace(go.Scatter(
                    mode="markers", x=[peak_pos], y=[peak_error[peak_pos-start_day]], 
                    marker_symbol="hexagram-dot", name="Momento do pico",
                    marker_line_color="midnightblue", marker_color="lightskyblue", 
                    marker_line_width=2, marker_size=15, 
                    hovertemplate="Pico no dia %{x} depois do começo da epidemia."))

fig1.update_layout(yaxis_range=[-1,31], 
                  template='xgridoff',
                  legend_orientation="h", legend=dict(x=0.20, y=1.0),
                  xaxis_title='Dias (desde o começo da epidemia) utilizados para treinamento',
                  yaxis_title='Erro (em dias) da estimativa do pico',
                  title_text="Erro da estimativa do pico - " + country['data']['name'])
# 'ggplot2', 'seaborn', 'simple_white', 'plotly',
# 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
# 'ygridoff', 'gridon', 'none'

fig1.show(renderer="png")
